In [1]:
import json
import pandas as pd

# Load the glossary data
records = []
with open("r1_glossary.jsonl", encoding="utf-8") as f:
    for line in f:
        entry = json.loads(line)
        records.append({
            "label": entry.get("label", ""),
            "id": entry.get("id", ""),
            "desc": entry.get("desc", "")
        })

# Convert to DataFrame and sort alphabetically by label (case-insensitive)
df = pd.DataFrame(records)
df_sorted = df.sort_values("label", key=lambda x: x.str.lower()).reset_index(drop=True)

# Save to CSV
df_sorted.to_csv("dbpedia_glossary_labels_sorted.csv", index=False)
print("Exported to dbpedia_glossary_labels_sorted.csv")


Exported to dbpedia_glossary_labels_sorted.csv


In [3]:
import pandas as pd
import requests
import rdflib
import time

def is_ascii(s): 
    return all(ord(c) < 128 for c in s)

df = pd.read_csv("dbpedia_glossary_labels_sorted.csv")

domains = []
ranges = []
comments = []

for idx, uri in enumerate(df['id']):
    print(f"Processing {idx+1}/{len(df)}: {uri}")
    if not uri.startswith("http://dbpedia.org/ontology/"):
        domains.append("")
        ranges.append("")
        comments.append("")
        continue

    ttl_url = uri + ".ttl"
    try:
        response = requests.get(ttl_url, timeout=10)
        if response.status_code != 200:
            domains.append("")
            ranges.append("")
            comments.append("")
            continue
        
        g = rdflib.Graph()
        g.parse(data=response.text, format="turtle")
        subj = rdflib.URIRef(uri)

        # Extract domain(s), range(s)
        d = [str(o) for o in g.objects(subj, rdflib.RDFS.domain)]
        r = [str(o) for o in g.objects(subj, rdflib.RDFS.range)]
        domains.append(", ".join(d) if d else "")
        ranges.append(", ".join(r) if r else "")

        # Extract and filter comment (ASCII only)
        c = [str(o) for o in g.objects(subj, rdflib.RDFS.comment)]
        comment_text = c[0] if c else ""
        if comment_text and is_ascii(comment_text):
            comments.append(comment_text)
        else:
            comments.append("")  # Empty if not ASCII/English-like

    except Exception as e:
        domains.append("")
        ranges.append("")
        comments.append("")

    time.sleep(0.2)  # polite pause

df['domain'] = domains
df['range'] = ranges
df['comment'] = comments

df.to_excel("dbpedia_glossary_with_domain_range_comment.xlsx", index=False)
print("Saved to dbpedia_glossary_with_domain_range_comment.xlsx")


Processing 1/2881: http://dbpedia.org/ontology/office
Processing 2/2881: http://dbpedia.org/ontology/municipalityRenamedTo
Processing 3/2881: http://dbpedia.org/ontology/presentName
Processing 4/2881: http://dbpedia.org/ontology/roleInEvent
Processing 5/2881: http://dbpedia.org/ontology/aSide
Processing 6/2881: http://dbpedia.org/ontology/abbeychurchBlessing
Processing 7/2881: http://dbpedia.org/ontology/abbeychurchBlessingCharge
Processing 8/2881: http://dbpedia.org/ontology/abbreviation
Processing 9/2881: http://dbpedia.org/ontology/ableToGrind
Processing 10/2881: http://dbpedia.org/ontology/absoluteMagnitude
Processing 11/2881: http://dbpedia.org/ontology/municipalityAbsorbedBy
Processing 12/2881: http://dbpedia.org/ontology/abstentions
Processing 13/2881: http://dbpedia.org/ontology/academicAdvisor
Processing 14/2881: http://dbpedia.org/ontology/academicDiscipline
Processing 15/2881: http://dbpedia.org/ontology/academyAward
Processing 16/2881: http://dbpedia.org/ontology/accelerati

In [12]:
import requests
from bs4 import BeautifulSoup

def extract_dbpedia_property(url, lang="en"):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the property metadata table by class
    property_table = soup.find('table', class_='table table-hover table-sm table-light')
    if not property_table:
        return {"label": "", "comment": "", "domain": "", "range": ""}

    label = comment = domain = range_ = ""
    for row in property_table.find_all('tr'):
        cells = row.find_all('td')
        if len(cells) < 2:
            continue
        key = cells[0].get_text(strip=True).lower()
        val_cell = cells[1]
        # For label and comment, prefer <span lang="en">
        if 'label' in key:
            en_span = val_cell.find('span', {'lang': lang})
            label = en_span.text.strip() if en_span else val_cell.get_text(strip=True)
        elif 'comment' in key:
            en_span = val_cell.find('span', {'lang': lang})
            comment = en_span.text.strip() if en_span else val_cell.get_text(strip=True)
        elif 'domain' in key:
            a_tag = val_cell.find('a')
            domain = a_tag.get_text(strip=True) if a_tag else val_cell.get_text(strip=True)
        elif 'range' in key:
            a_tag = val_cell.find('a')
            range_ = a_tag.get_text(strip=True) if a_tag else val_cell.get_text(strip=True)
    return {"label": label, "comment": comment, "domain": domain, "range": range_}

# Test for /director
url = "https://dbpedia.org/ontology/director"
print(extract_dbpedia_property(url))


{'label': 'film director', 'comment': 'A film director is a person who directs the making of a film.', 'domain': 'dbo:Film', 'range': 'dbo:Person'}


In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

def extract_dbpedia_property(url, lang="en"):
    try:
        response = requests.get(url, timeout=20)
        soup = BeautifulSoup(response.text, 'html.parser')
        property_table = soup.find('table', class_='table table-hover table-sm table-light')
        if not property_table:
            return {"label": "", "comment": "", "domain": "", "range": ""}
        label = comment = domain = range_ = ""
        for row in property_table.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) < 2:
                continue
            key = cells[0].get_text(strip=True).lower()
            val_cell = cells[1]
            # For label and comment, prefer <span lang="en">
            if 'label' in key:
                en_span = val_cell.find('span', {'lang': lang})
                label = en_span.text.strip() if en_span else val_cell.get_text(strip=True)
            elif 'comment' in key:
                en_span = val_cell.find('span', {'lang': lang})
                comment = en_span.text.strip() if en_span else val_cell.get_text(strip=True)
            elif 'domain' in key:
                a_tag = val_cell.find('a')
                domain = a_tag.get_text(strip=True) if a_tag else val_cell.get_text(strip=True)
            elif 'range' in key:
                a_tag = val_cell.find('a')
                range_ = a_tag.get_text(strip=True) if a_tag else val_cell.get_text(strip=True)
        return {"label": label, "comment": comment, "domain": domain, "range": range_}
    except Exception as e:
        print(f"Error with {url}: {e}")
        return {"label": "", "comment": "", "domain": "", "range": ""}

# Read your CSV (column "id" should have the property URLs)
df = pd.read_csv("dbpedia_glossary_labels_sorted.csv")

results = []

# Process ALL rows 
for idx, row in df.iterrows():
    url = row['id']
    print(f"Processing {idx+1}/{len(df)}: {url}")
    meta = extract_dbpedia_property(url)
    result = {
        "id": url,
        "label": meta["label"],
        "comment": meta["comment"],
        "domain": meta["domain"],
        "range": meta["range"]
    }
    results.append(result)
    time.sleep(0.3)  # Polite pause (you may increase to 0.5 for very large files)

results_df = pd.DataFrame(results)
results_df.to_excel("dbpedia_metadata_first10.xlsx", index=False)
print("Saved to dbpedia_metadata_first10.xlsx")


Processing 1/2881: http://dbpedia.org/ontology/office
Processing 2/2881: http://dbpedia.org/ontology/municipalityRenamedTo
Processing 3/2881: http://dbpedia.org/ontology/presentName
Processing 4/2881: http://dbpedia.org/ontology/roleInEvent
Processing 5/2881: http://dbpedia.org/ontology/aSide
Processing 6/2881: http://dbpedia.org/ontology/abbeychurchBlessing
Processing 7/2881: http://dbpedia.org/ontology/abbeychurchBlessingCharge
Processing 8/2881: http://dbpedia.org/ontology/abbreviation
Processing 9/2881: http://dbpedia.org/ontology/ableToGrind
Processing 10/2881: http://dbpedia.org/ontology/absoluteMagnitude
Processing 11/2881: http://dbpedia.org/ontology/municipalityAbsorbedBy
Processing 12/2881: http://dbpedia.org/ontology/abstentions
Processing 13/2881: http://dbpedia.org/ontology/academicAdvisor
Processing 14/2881: http://dbpedia.org/ontology/academicDiscipline
Processing 15/2881: http://dbpedia.org/ontology/academyAward
Processing 16/2881: http://dbpedia.org/ontology/accelerati

Final

In [21]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

def is_en_pt(text):
    # Accept English, Portuguese, numbers, common punctuation, spaces
    return bool(re.match(r'^[a-zA-Z0-9 áéíóúãõâêîôûçÁÉÍÓÚÃÕÂÊÎÔÛÇ.,;:\'\"!\?\-\(\)/_]+$', str(text)))

def extract_dbpedia_property(url, lang="en"):
    try:
        response = requests.get(url, timeout=20)
        soup = BeautifulSoup(response.text, 'html.parser')
        property_table = soup.find('table', class_='table table-hover table-sm table-light')
        if not property_table:
            return {"label": "", "comment": "", "domain": "", "range": ""}
        label = comment = domain = range_ = ""
        for row in property_table.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) < 2:
                continue
            key = cells[0].get_text(strip=True)
            val_cell = cells[1]
            if key == "rdfs:label":
                en_span = val_cell.find('span', {'lang': lang})
                val = en_span.text.strip() if en_span else val_cell.get_text(strip=True)
                if not is_en_pt(val):
                    print(f"⚠️ Non-English/Portuguese LABEL for {url}: {val}")
                    val = ""  # or "NON-ENGLISH CONTENT"
                label = val
            elif key == "rdfs:comment":
                en_span = val_cell.find('span', {'lang': lang})
                val = en_span.text.strip() if en_span else val_cell.get_text(strip=True)
                if not is_en_pt(val):
                    print(f"⚠️ Non-English/Portuguese COMMENT for {url}: {val}")
                    val = ""  # or "NON-ENGLISH CONTENT"
                comment = val
            elif key == "rdfs:domain":
                a_tag = val_cell.find('a')
                domain = a_tag.get_text(strip=True) if a_tag else val_cell.get_text(strip=True)
            elif key == "rdfs:range":
                a_tag = val_cell.find('a')
                range_ = a_tag.get_text(strip=True) if a_tag else val_cell.get_text(strip=True)
        return {"label": label, "comment": comment, "domain": domain, "range": range_}
    except Exception as e:
        print(f"Error with {url}: {e}")
        return {"label": "", "comment": "", "domain": "", "range": ""}

# Read your CSV (should have 'id', 'label', and 'desc' columns)
df = pd.read_csv("dbpedia_glossary_labels_sorted.csv")
results = []

for idx, row in df.iterrows():
    url = row['id']
    original_desc = row.get('desc', "")
    print(f"Processing {idx+1}/{len(df)}: {url}")
    meta = extract_dbpedia_property(url)
    # If comment from online page is empty, use original desc as fallback
    comment_final = meta["comment"] if meta["comment"] else original_desc
    result = {
        "id": url,
        "label": meta["label"],
        "comment": comment_final,
        "domain": meta["domain"],
        "range": meta["range"],
        "desc_original": original_desc
    }
    results.append(result)
    time.sleep(0.3)

results_df = pd.DataFrame(results)
results_df.to_excel("dbpedia_metadata_final_with_desc.xlsx", index=False)
print("Saved to dbpedia_metadata_final_with_desc.xlsx")


Processing 1/2881: http://dbpedia.org/ontology/office
Processing 2/2881: http://dbpedia.org/ontology/municipalityRenamedTo
Processing 3/2881: http://dbpedia.org/ontology/presentName
Processing 4/2881: http://dbpedia.org/ontology/roleInEvent
Processing 5/2881: http://dbpedia.org/ontology/aSide
Processing 6/2881: http://dbpedia.org/ontology/abbeychurchBlessing
Processing 7/2881: http://dbpedia.org/ontology/abbeychurchBlessingCharge
Processing 8/2881: http://dbpedia.org/ontology/abbreviation
Processing 9/2881: http://dbpedia.org/ontology/ableToGrind
Processing 10/2881: http://dbpedia.org/ontology/absoluteMagnitude
Processing 11/2881: http://dbpedia.org/ontology/municipalityAbsorbedBy
Processing 12/2881: http://dbpedia.org/ontology/abstentions
Processing 13/2881: http://dbpedia.org/ontology/academicAdvisor
Processing 14/2881: http://dbpedia.org/ontology/academicDiscipline
Processing 15/2881: http://dbpedia.org/ontology/academyAward
⚠️ Non-English/Portuguese COMMENT for http://dbpedia.org/o

Clean the not expected characters from desc

In [23]:
import pandas as pd
import re

def is_en_pt(text):
    # Accept English, Portuguese, numbers, punctuation, spaces
    if pd.isna(text):
        return True  # Consider NaN as OK
    return bool(re.match(r'^[a-zA-Z0-9 áéíóúãõâêîôûçÁÉÍÓÚÃÕÂÊÎÔÛÇ.,;:\'\"!\?\-\(\)/_]+$', str(text)))

# Load the output file
df = pd.read_excel("dbpedia_metadata_final_with_desc.xlsx")

# Clean the comment column
for i, row in df.iterrows():
    comment = row['comment']
    if not is_en_pt(comment):
        print(f"⚠️ Non-English/Portuguese COMMENT at row {i}: {comment}")
        df.at[i, 'comment'] = ""

# Optionally clean label or any other text column
# for i, row in df.iterrows():
#     label = row['label']
#     if not is_en_pt(label):
#         print(f"⚠️ Non-English/Portuguese LABEL at row {i}: {label}")
#         df.at[i, 'label'] = ""

df.to_excel("dbpedia_metadata_final_with_desc_cleaned.xlsx", index=False)
print("Saved to dbpedia_metadata_final_with_desc_cleaned.xlsx")


⚠️ Non-English/Portuguese COMMENT at row 21: وبائی مرض میں فعال کیسوں کی تعداد
⚠️ Non-English/Portuguese COMMENT at row 121: apoapsis (μ)
⚠️ Non-English/Portuguese COMMENT at row 168: asset under management ($)
⚠️ Non-English/Portuguese COMMENT at row 180: 阿斯利康制药公司，是一家由瑞典阿斯特公司（Astra AB）和英国捷利康公司（Zeneca Group PLC）于1999年4月6日合并而成的大型英瑞合资生物制药企业
⚠️ Non-English/Portuguese COMMENT at row 187: the ratio of the average mass of atoms of an element (from a single given sample or source) to 1⁄12 of the mass of an atom of carbon-12
⚠️ Non-English/Portuguese COMMENT at row 188: the ratio of the average mass of atoms of an element (from a single given sample or source) to 1⁄12 of the mass of an atom of carbon-12
⚠️ Non-English/Portuguese COMMENT at row 211: used for DBpedia History > Subject must be a blank node containing a dc:date and a rdfs:value
⚠️ Non-English/Portuguese COMMENT at row 212: used for DBpedia History > Subject must be a blank node containing a dc:date and a rdfs:value
⚠️ Non-English/

In [30]:
import pandas as pd
import re

def is_en_pt(text):
    if pd.isna(text):
        return True
    text = str(text)
    allowed_extras = "áéíóúãõâêîôûçÁÉÍÓÚÃÕÂÊÎÔÛÇ£µ³²°⁄μ-’_–"  # Add more as needed
    for char in text:
        code = ord(char)
        if (
            32 <= code <= 126              # ASCII printable
            or char in allowed_extras      # Accented/special letters, μ, µ, £, etc.
            or char.isspace()
        ):
            continue
        else:
            return False
    return True

# Load your file
df = pd.read_excel("dbpedia_metadata_final_with_desc.xlsx")

for i, row in df.iterrows():
    comment = row['comment']
    if not is_en_pt(comment):
        print(f"⚠️ Non-English/Portuguese COMMENT at row {i}: {comment}")
        df.at[i, 'comment'] = ""

df.to_excel("dbpedia_metadata_final_with_desc_cleaned.xlsx", index=False)
print("Saved to dbpedia_metadata_final_with_desc_cleaned.xlsx")


⚠️ Non-English/Portuguese COMMENT at row 21: وبائی مرض میں فعال کیسوں کی تعداد
⚠️ Non-English/Portuguese COMMENT at row 180: 阿斯利康制药公司，是一家由瑞典阿斯特公司（Astra AB）和英国捷利康公司（Zeneca Group PLC）于1999年4月6日合并而成的大型英瑞合资生物制药企业
⚠️ Non-English/Portuguese COMMENT at row 975: Ein Gründer oder Gründungsmitglied einer Organisation, Religion oder eines Ortes.
⚠️ Non-English/Portuguese COMMENT at row 1566: 莫德纳，是一家总部位于美国马萨诸塞州剑桥市的跨国制药、生物技术公司，专注于癌症免疫治疗，包括基于mRNA的药物发现、药物研发和疫苗技术
⚠️ Non-English/Portuguese COMMENT at row 2021: 辉瑞是源自美国的跨国制药、生物技术公司，营运总部位于纽约，研发总部位于康涅狄格州的格罗顿市
⚠️ Non-English/Portuguese COMMENT at row 2571: Système universitaire de documentation id (French collaborative library catalog).
http://www.idref.fr/$1
⚠️ Non-English/Portuguese COMMENT at row 2697: Scale parameters that can be understood by Geohack, eg "type:", "scale:", "region:" "altitude:". Use "_" for several (eg "type:landmark_scale:50000"). See https://fr.wikipedia.org/wiki/Modèle:Infobox_Subdivision_administrative for examples, and https://fr.

Frequency distribution of domain and range

In [32]:
import pandas as pd

# Load your cleaned file
df = pd.read_excel("dbpedia_metadata_final_with_desc_cleaned.xlsx")

# Frequency distribution for domain
domain_counts = df['domain'].value_counts(dropna=False).reset_index()
domain_counts.columns = ['domain', 'count']

# Frequency distribution for range
range_counts = df['range'].value_counts(dropna=False).reset_index()
range_counts.columns = ['range', 'count']

# Show top 20 for each (or as needed)
print("=== Domain Frequency Distribution ===")
print(domain_counts.head(20))

print("\n=== Range Frequency Distribution ===")
print(range_counts.head(20))

# Optionally save to Excel
with pd.ExcelWriter("dbpedia_domain_range_frequencies.xlsx") as writer:
    domain_counts.to_excel(writer, sheet_name='domain', index=False)
    range_counts.to_excel(writer, sheet_name='range', index=False)

print("\nSaved to dbpedia_domain_range_frequencies.xlsx")


=== Domain Frequency Distribution ===
                        domain  count
0                          NaN    526
1                   dbo:Person    235
2                    dbo:Place    185
3           dbo:PopulatedPlace    149
4                  dbo:Athlete     65
5               dbo:Settlement     56
6                   dbo:Island     36
7                     dbo:Work     35
8             dbo:SpaceMission     34
9             dbo:TennisPlayer     33
10                  dbo:School     33
11            dbo:Organisation     31
12            dbo:MilitaryUnit     29
13                 dbo:Species     29
14                 dbo:Disease     26
15       dbo:ChemicalSubstance     25
16             dbo:Broadcaster     25
17                    dbo:Film     24
18                  dbo:Artist     23
19  dbo:ArchitecturalStructure     23

=== Range Frequency Distribution ===
                       range  count
0                 xsd:string    811
1     xsd:nonNegativeInteger    301
2                 

Find Messiness

In [34]:
import pandas as pd

# Load your cleaned file
df = pd.read_excel("dbpedia_metadata.xlsx")

def type_counts(column, prefix):
    """Returns the count and percentage for a prefix in a column."""
    col = df[column].fillna("")
    match = col.str.startswith(prefix)
    count = match.sum()
    pct = 100 * count / len(col)
    return count, pct

def is_blank(val):
    return pd.isna(val) or str(val).strip() == ""

total = len(df)

# Blank/NaN counts
blank_domain = df['domain'].isna().sum() + (df['domain'].astype(str).str.strip() == "").sum()
blank_range = df['range'].isna().sum() + (df['range'].astype(str).str.strip() == "").sum()

# Range types
entity_range_count = df['range'].fillna("").str.startswith(("dbo:", "rdf:")).sum()
literal_range_count = df['range'].fillna("").str.startswith("xsd:").sum()
other_range_count = total - blank_range - entity_range_count - literal_range_count

print(f"Total rows: {total}")
print("\n--- DOMAIN ---")
print(f"Blank/NaN: {blank_domain} ({blank_domain/total:.1%})")
print("\n--- RANGE ---")
print(f"Blank/NaN: {blank_range} ({blank_range/total:.1%})")
print(f"Entity type (starts with dbo:/rdf:): {entity_range_count} ({entity_range_count/total:.1%})")
print(f"Literal/datatype (starts with xsd:): {literal_range_count} ({literal_range_count/total:.1%})")
print(f"Other: {other_range_count} ({other_range_count/total:.1%})")

# Optional: Print some examples of "other"
if other_range_count > 0:
    print("\nExamples of 'other' range types:")
    others = df[~df['range'].fillna("").str.startswith(("dbo:", "rdf:", "xsd:")) & (df['range'].fillna("").str.strip() != "")]
    print(others['range'].drop_duplicates().head(10).tolist())


Total rows: 2881

--- DOMAIN ---
Blank/NaN: 526 (18.3%)

--- RANGE ---
Blank/NaN: 290 (10.1%)
Entity type (starts with dbo:/rdf:): 919 (31.9%)
Literal/datatype (starts with xsd:): 1652 (57.3%)
Other: 20 (0.7%)

Examples of 'other' range types:
['skos:Concept', 'geo:SpatialThing', 'dbd:engineConfiguration', 'skos:OrderedCollection', 'dbd:valvetrain', 'dbd:hectopascal']
